In [3]:
import pandas as pd
from influxdb import InfluxDBClient

import json
import re
from datetime import datetime
import requests

In [4]:
#optional, jupyter notebook dislay options to see more data
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None
pd.options.display.precision = 15

In [5]:
coin_auth = json.load(open('coin_credentials'))
coin_client = InfluxDBClient(
    host='testresults.qt.io', 
    username=coin_auth['username'],
    password=coin_auth['password'],
    port=8086,
    database='coin')

In [57]:
#q = "select * from flaky_test where branch = 'dev' and time > now() - 2d"
#alternative queries:
q = "select * from flaky_test where branch = 'dev' and time > now() - 5d"
df = pd.DataFrame(coin_client.query(q).get_points())
#since we do not have stats about MacOS we get won't analyse those tests
df.drop(df[df.target_os == 'MacOS'].index, inplace=True)
#some pre-processing to group the tests by : testcase, testfunction and platform on wich it was run:
df['platform'] = df['target_os_version'] + '+' + df['target_arch']+  '+' + df['target_compiler']
df['full_test_params'] = df['project'] + '+'+ df['testCase'] + '+' + df['testFunction'] +'+' + df['platform']
df.head()

,time,branch,computeHost,cpu_count,cpu_count_1,cpu_seconds,duration,host_arch,host_compiler,host_os,host_os_version,id,path,platform_id,project,result,sha1,state,target_arch,target_compiler,target_os,target_os_version,taskType,testCase,testFunction,type,platform,full_test_params
16,2023-01-13T14:46:34.322202Z,dev,big-grouse.on1.qt.io,4,None,16184,4046,X86_64,GCC,Linux,openSUSE_15_4,1681437589,qt/qtbase/7a7cdf12cba1efd6ba8dbf8d632100369a59fc7d/LinuxopenSUSE_15_4x86_64LinuxopenSUSE_15_4x86_64GCCqtci-linux-openSUSE-15.4-x86_64-50-11fcc3Sccache_UseConfigure_WarningsAreErrors/6d298dc4711b0382097b2ef17e67ff5d9ca31816/test_1681437597,opensuse-15.4-developer-build,qt/qtbase,Failed,984e2ebec4caf266de6efa45ade26ac72bd6f282,Done,X86_64,GCC,Linux,openSUSE_15_4,Integration,tst_qgesturerecognizer,panGesture,Test,openSUSE_15_4+X86_64+GCC,qt/qtbase+tst_qgesturerecognizer+panGesture+openSUSE_15_4+X86_64+GCC
17,2023-01-13T14:46:34.322202Z,dev,big-grouse.on1.qt.io,4,None,16184,4046,X86_64,GCC,Linux,openSUSE_15_4,1681437589,qt/qtbase/7a7cdf12cba1efd6ba8dbf8d632100369a59fc7d/LinuxopenSUSE_15_4x86_64LinuxopenSUSE_15_4x86_64GCCqtci-linux-openSUSE-15.4-x86_64-50-11fcc3Sccache_UseConfigure_WarningsAreErrors/6d298dc4711b0382097b2ef17e67ff5d9ca31816/test_1681437597,opensuse-15.4-developer-build,qt/qtbase,Failed,984e2ebec4caf266de6efa45ade26ac72bd6f282,Done,X86_64,GCC,Linux,openSUSE_15_4,Integration,tst_qtcpserver,ipv6Server,Test,openSUSE_15_4+X86_64+GCC,qt/qtbase+tst_qtcpserver+ipv6Server+openSUSE_15_4+X86_64+GCC
18,2023-01-13T14:46:34.322202Z,dev,pinky-pig.on1.qt.io,4,None,11684,2921,X86_64,GCC,Linux,Ubuntu_20_04,1681437589,qt/qtbase/7a7cdf12cba1efd6ba8dbf8d632100369a59fc7d/LinuxUbuntu_20_04x86_64LinuxQEMUarmv7GCCqtci-linux-Ubuntu-20.04-x86_64-50-a897c8Sccache_UseConfigure/e2a8c111ddc35c08339e6abe7df6be265c38555c/test_1681437631,qemu-armv7-developer-build,qt/qtbase,Failed,984e2ebec4caf266de6efa45ade26ac72bd6f282,Done,ARMv7,GCC,Linux,QEMU,Integration,tst_qnetworkreply,qtbug12908compressedHttpReply,Test,QEMU+ARMv7+GCC,qt/qtbase+tst_qnetworkreply+qtbug12908compressedHttpReply+QEMU+ARMv7+GCC
42,2023-01-13T14:54:32.489036Z,dev,calm-squid.on1.qt.io,2,None,1118,559,X86_64,MSVC2019,Windows,Windows_11_22H2,1681438943,qt/qttools/e038f1448503797f5f0a24dec74fb03ac59746cf/WindowsWindows_11_22H2x86_64WindowsWindows_11_22H2x86_64MSVC2019qtci-windows-11_22H2-x86_64-50-8a755cDebugAndRelease_Sccache_UseConfigure_WarningsAreErrors/6c7cc52bb21e46a633e7c2c5b187e41d3168a48e/test_1681438966,windows-11_22h2-msvc2019-developer-build,qt/qttools,Failed,c925d1b4a4e29e9994db004b215fda01067562e6,Done,X86_64,MSVC2019,Windows,Windows_11_22H2,Integration,tst_lupdate,good,Test,Windows_11_22H2+X86_64+MSVC2019,qt/qttools+tst_lupdate+good+Windows_11_22H2+X86_64+MSVC2019
43,2023-01-13T14:54:32.489036Z,dev,grand-mako.on1.qt.io,2,None,1232,616,X86_64,MSVC2022,Windows,Windows_11_22H2,1681438943,qt/qttools/e038f1448503797f5f0a24dec74fb03ac59746cf/WindowsWindows_11_22H2x86_64WindowsWindows_11_22H2x86_64MSVC2022qtci-windows-11_22H2-x86_64-50-8a755cDebugAndRelease_Sccache_UseConfigure/cb6d4e0c3513b5562fb735f349958ee2cd1b0969/test_1681438960,windows-11_22h2-msvc2022,qt/qttools,Failed,c925d1b4a4e29e9994db004b215fda01067562e6,Done,X86_64,MSVC2022,Windows,Windows_11_22H2,Integration,tst_lupdate,good,Test,Windows_11_22H2+X86_64+MSVC2022,qt/qttools+tst_lupdate+good+Windows_11_22H2+X86_64+MSVC2022


In [58]:
df['full_test_params']
df2 = df.groupby(['full_test_params'])['full_test_params'].count().to_frame()
df2 = df2.rename(columns={"full_test_params": "count"})
df2.sort_values(by=['count'], inplace = True, ascending = False)
df2.reset_index(inplace=True)
df2.head()


,full_test_params,count
0,qt/qtbase+tst_qwidget+enterLeaveOnWindowShowHide+Windows_11_22H2+X86_64+MSVC2019,52
1,qt/qtwebengine+tst_qwebenginepage+backgroundColor+Windows_10_22H2+X86_64+MSVC2019,38
2,qt/qtdeclarative+tst_qquicktextarea+touchscreenDoesNotSelect+Android_ANY+X86+Clang,31
3,qt/qtwebengine+tst_qwebengineprofile+clearDataFromCache+Windows_11_22H2+X86_64+MSVC2019,30
4,qt/qtbase+tst_qgesturerecognizer+panGesture+openSUSE_15_4+X86_64+GCC,28


In [60]:
res = df2.iloc[0]['full_test_params'].split("+")
project = res[0]
target_os_version = res[3]
target_arch = res[4]
target_compiler = res[5]
project, target_os_version, target_arch, target_compiler

('qt/qtbase', 'Windows_11_22H2', 'X86_64', 'MSVC2019')

# fidning passed workitems data

In [61]:
#q = "select * from flaky_test where branch = 'dev' and time > now() - 2d"
#alternative queries:
q = "select * from workitem where branch = 'dev' and project = '{}' and target_os_version='{}' and target_arch='{}' and target_compiler='{}' \
and type = 'Test' and state = 'Done' and time > now() - 5d".format(project, target_os_version, target_arch, target_compiler)
print(q)
df4 = pd.DataFrame(coin_client.query(q).get_points())
df4.head()

select * from workitem where branch = 'dev' and project = 'qt/qtbase' and target_os_version='Windows_11_22H2' and target_arch='X86_64' and target_compiler='MSVC2019' and type = 'Test' and state = 'Done' and time > now() - 5d


,time,branch,computeHost,cpu_count,cpu_count_1,cpu_seconds,duration,host_arch,host_compiler,host_os,host_os_version,id,path,platform_id,project,sha1,state,target_arch,target_compiler,target_os,target_os_version,taskType,type
0,2023-01-13T14:53:44Z,dev,needed-wasp.on1.qt.io,4,None,12980,3245,X86_64,MSVC2019,Windows,Windows_11_22H2,1681443059,qt/qtbase/351e5fa6710d29a10fcb025b05c8ceacee960dc3/WindowsWindows_11_22H2x86_64WindowsWindows_11_22H2x86_64MSVC2019qtci-windows-11_22H2-x86_64-50-8a755cDebugAndRelease_Sccache_UseConfigure/ed5cab89f998c6de21d8b249b54f3d2636e52b0f/test_1681443076,windows-11_22h2-msvc2019,qt/qtbase,3f907e3d464d3678d2a5abde5787b3982e28d4c9,Done,X86_64,MSVC2019,Windows,Windows_11_22H2,Integration,Test
1,2023-01-13T15:25:35Z,dev,on-ox.on1.qt.io,4,None,15784,3946,X86_64,MSVC2019,Windows,Windows_11_22H2,1681443059,qt/qtbase/351e5fa6710d29a10fcb025b05c8ceacee960dc3/WindowsWindows_11_22H2x86_64WindowsWindows_11_22H2x86_64MSVC2019qtci-windows-11_22H2-x86_64-50-8a755cDebugAndRelease_Sccache_UseConfigure_WarningsAreErrors/3f1ea717f4de242a457aff7bf9f20895b7f07b16/test_1681443086,windows-11_22h2-msvc2019-developer-build,qt/qtbase,3f907e3d464d3678d2a5abde5787b3982e28d4c9,Done,X86_64,MSVC2019,Windows,Windows_11_22H2,Integration,Test
2,2023-01-13T16:12:44Z,dev,on-ox.on1.qt.io,4,None,13480,3370,X86_64,MSVC2019,Windows,Windows_11_22H2,1681444443,qt/qtbase/ade199a4857026674abefc56b3d6c8853cb5de90/WindowsWindows_11_22H2x86_64WindowsWindows_11_22H2x86_64MSVC2019qtci-windows-11_22H2-x86_64-50-8a755cDebugAndRelease_Sccache_UseConfigure/ed5cab89f998c6de21d8b249b54f3d2636e52b0f/test_1681444456,windows-11_22h2-msvc2019,qt/qtbase,adf54f49847f7fec8d77a3409277b2beed549613,Done,X86_64,MSVC2019,Windows,Windows_11_22H2,Integration,Test
3,2023-01-13T17:02:46Z,dev,big-grouse.on1.qt.io,4,None,17608,4402,X86_64,MSVC2019,Windows,Windows_11_22H2,1681444443,qt/qtbase/ade199a4857026674abefc56b3d6c8853cb5de90/WindowsWindows_11_22H2x86_64WindowsWindows_11_22H2x86_64MSVC2019qtci-windows-11_22H2-x86_64-50-8a755cDebugAndRelease_Sccache_UseConfigure_WarningsAreErrors/3f1ea717f4de242a457aff7bf9f20895b7f07b16/test_1681444466,windows-11_22h2-msvc2019-developer-build,qt/qtbase,adf54f49847f7fec8d77a3409277b2beed549613,Done,X86_64,MSVC2019,Windows,Windows_11_22H2,Integration,Test
4,2023-01-13T17:12:10Z,dev,disappointed-cat.on1.qt.io,4,None,9576,2394,X86_64,MSVC2019,Windows,Windows_11_22H2,1681450304,qt/qtbase/ab0f703d6af7ce769bd0e3c9831eb19b1d12125a/WindowsWindows_11_22H2x86_64WindowsWindows_11_22H2x86_64MSVC2019qtci-windows-11_22H2-x86_64-50-8a755cDebugAndRelease_Sccache_UseConfigure/ed5cab89f998c6de21d8b249b54f3d2636e52b0f/test_1681450347,windows-11_22h2-msvc2019,qt/qtbase,f948e52ed224693f5d78a155cabef1e6ea21f8e4,Done,X86_64,MSVC2019,Windows,Windows_11_22H2,Integration,Test


In [62]:
df4.shape

(129, 23)

# excluding workitems that had some flaky history for the test we are checking

In [63]:
 df2.iloc[1]['full_test_params']

'qt/qtwebengine+tst_qwebenginepage+backgroundColor+Windows_10_22H2+X86_64+MSVC2019'

In [64]:
checked_testcase = df2.iloc[0]['full_test_params']
checked_testcase
df_to_exclude = df[df['full_test_params'] == checked_testcase]
checked_testcase, df_to_exclude.shape, df4.shape

('qt/qtbase+tst_qwidget+enterLeaveOnWindowShowHide+Windows_11_22H2+X86_64+MSVC2019',
 (52, 28),
 (129, 23))

In [65]:
ids_to_exlude = df_to_exclude.id.unique()
ids_to_exlude

array(['web_qt_qtbase_1673613638092', '1681443059', '1681450439',
       '1681455027', '1681439320', 'nightly1681456073', '1681461015',
       '1681462512', '1681462810', 'nightly1681471017', '1681477116',
       'web_qt_qtbase_1673857201813', '1681488450',
       'web_qt_qt5_1673856387504', '1681492351', '1681492162',
       '1681500013', '1681487503', '1681501278', '1681502169',
       'web_qt_qt5_1673881189963', '1681503746', '1681509133',
       '1681509638', '1681515337', '1681517044', '1681530484',
       '1681527814', '1681527240', '1681529024', '1681533666',
       '1681530493', '1681530505', '1681530374', '1681551474',
       '1681535506', '1681560201', '1681560369', '1681566923',
       'nightly1681555073', '1681566407', '1681571494', '1681573908',
       '1681574470'], dtype=object)

In [66]:
good_workitems_df = df4[~df4['id'].isin(ids_to_exlude)]
good_workitems_df.shape

(60, 23)

# fidning timestamp when test passed (from log.txt.gz)


In [67]:
#just one example can be iterated
path = good_workitems_df.iloc[0]['path']
url = "https://testresults.qt.io/logs/{}/log.txt.gz".format(path)
#url is path to log.txt.gz with test execution data, where link to telegraf metrics is added
response = requests.get(url)
response, url

(<Response [200]>,
 'https://testresults.qt.io/logs/qt/qtbase/ade199a4857026674abefc56b3d6c8853cb5de90/WindowsWindows_11_22H2x86_64WindowsWindows_11_22H2x86_64MSVC2019qtci-windows-11_22H2-x86_64-50-8a755cDebugAndRelease_Sccache_UseConfigure/ed5cab89f998c6de21d8b249b54f3d2636e52b0f/test_1681444456/log.txt.gz')

In [71]:
res = checked_testcase.split('+')
res

['qt/qtbase',
 'tst_qwidget',
 'enterLeaveOnWindowShowHide',
 'Windows_11_22H2',
 'X86_64',
 'MSVC2019']

In [73]:
searched_text = "PASS   : {}::{}".format(res[1],res[2] )
searched_text


'PASS   : tst_qwidget::enterLeaveOnWindowShowHide'

In [74]:
for line in response.text.split('\n'):
    if searched_text.lower() in line.lower():
        break
line

'agent:2023/01/13 17:01:56 build.go:403: 469: PASS   : tst_QWidget::enterLeaveOnWindowShowHide(dialog)'

In [75]:
timestamp = line.split(" build.go")[0]
timestamp_str  = timestamp.split("agent:")[1]
timestamp_str

'2023/01/13 17:01:56'

In [76]:
import time
timestamp_str = '2023/01/13 13:00:36'
datetime_timestamp = datetime.strptime(timestamp_str, '%Y/%m/%d %H:%M:%S')
datetime_timestamp, time.mktime(datetime_timestamp.timetuple())

(datetime.datetime(2023, 1, 13, 13, 0, 36), 1673611236.0)

# customize this code!!! connecting  given test data with vm data 

In [77]:
#just taking first random test:
path = df.iloc[0]['path']
url = "https://testresults.qt.io/logs/{}/log.txt.gz".format(path)
#url is path to log.txt.gz with test execution data, where link to telegraf metrics is added
response = requests.get(url)
response

<Response [200]>

In [6]:
#ugly processing of respone to find the start timestamp. end timestamp. and server id
#I search for:
#INFO:workitem.py:760(23870): <a href="https://inframetrics.intra.qt.io/d/nOAsINmZz/telegraf-coin-vms?orgId=1&from=1673829274000&to=1673830968000&var-server=1681269718-122700">VM metrics
result = re.findall(r"INFO:workitem.py:[0-9]+\([0-9]+\):.+VM metrics", response.text)
result
start_time = re.findall(r"from=[0-9]+", result[0])
start_timestamp = start_time[0].split('from=')[1]
end_time = re.findall(r"to=[0-9]+", result[0])
end_timestamp = end_time[0].split('to=')[1]
server_id = re.findall(r"server=[0-9]+-[0-9]+", result[0])
server_id = server_id[0].split('server=')[1]
#start_timestamp is in Unix seconds, you can convert it to human readable time
start_timestamp, end_timestamp, server_id

('1673862903000', '1673863535000', '1681269718-129030')

In [7]:
#connecting to telegraf
telegraf_auth = json.load(open('telegraf_credentials'))
telegraf_client = InfluxDBClient(
    host="qtinfluxdb01.intra.qt.io",
    username=telegraf_auth['username'],
    password=telegraf_auth['password'],
    port=8086,
    ssl=True,
    database="coin_vms")

In [8]:
#all vm mesurements we keep
q = "show measurements"
tdf = pd.DataFrame(telegraf_client.query(q).get_points())
tdf

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,name
0,cpu
1,disk
2,diskio
3,ioping
4,kernel
5,kernel_vmstat
6,mem
7,netstat
8,processes
9,procstat_lookup


In [9]:
#example query
q = "SELECT *  FROM mem WHERE host =~ /{}(-|$)/ AND time >= {}ms and time <= {}ms".format(server_id,start_timestamp, end_timestamp)
tdf = pd.DataFrame(telegraf_client.query(q).get_points())
tdf.head()

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,time,active,available,available_percent,buffered,cached,commit_limit,committed_as,dirty,free,high_free,high_total,host,huge_page_size,huge_pages_free,huge_pages_total,inactive,low_free,low_total,mapped,page_tables,shared,slab,swap_cached,swap_free,swap_total,total,used,used_percent,vmalloc_chunk,vmalloc_total,vmalloc_used,wired,write_back,write_back_tmp
0,2023-01-16T09:55:20Z,0,7898918912,81.742812284351572,0,0,0,0,0,0,0,0,1681269718-129030-Windows_10_22H2,0,0,0,0,0,0,0,0,0,0,0,0,0,9663135744,1764216832,18.257187715648424,0,0,0,0,0,0
1,2023-01-16T09:55:30Z,0,7518089216,77.801755198027777,0,0,0,0,0,0,0,0,1681269718-129030-Windows_10_22H2,0,0,0,0,0,0,0,0,0,0,0,0,0,9663135744,2145046528,22.198244801972223,0,0,0,0,0,0
2,2023-01-16T09:55:40Z,0,7617515520,78.830679003240135,0,0,0,0,0,0,0,0,1681269718-129030-Windows_10_22H2,0,0,0,0,0,0,0,0,0,0,0,0,0,9663135744,2045620224,21.169320996759868,0,0,0,0,0,0
3,2023-01-16T09:55:50Z,0,7438950400,76.982778645316728,0,0,0,0,0,0,0,0,1681269718-129030-Windows_10_22H2,0,0,0,0,0,0,0,0,0,0,0,0,0,9663135744,2224185344,23.017221354683269,0,0,0,0,0,0
4,2023-01-16T09:56:00Z,0,7381303296,76.386211386745472,0,0,0,0,0,0,0,0,1681269718-129030-Windows_10_22H2,0,0,0,0,0,0,0,0,0,0,0,0,0,9663135744,2281832448,23.613788613254524,0,0,0,0,0,0


In [10]:
#how many datapoints there are for give time slice (start_timestamp, end_timestamp))
tdf.shape

(62, 35)

In [11]:
q = "SELECT *  FROM cpu WHERE host =~ /{}(-|$)/ AND time >= {}ms and time <= {}ms".format(server_id,start_timestamp, end_timestamp)

tdf = pd.DataFrame(telegraf_client.query(q).get_points())
tdf.head()

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,time,cpu,host,usage_guest,usage_guest_nice,usage_idle,usage_iowait,usage_irq,usage_nice,usage_softirq,usage_steal,usage_system,usage_user
0,2023-01-16T09:55:30Z,cpu-total,1681269718-129030-Windows_10_22H2,0.0,0.0,33.520900321543408,0.0,0.000000000000000,0.0,0.0,0.0,21.704180064308680,44.774919614147912
1,2023-01-16T09:55:30Z,cpu0,1681269718-129030-Windows_10_22H2,0.0,0.0,31.093750000000000,0.0,0.000000000000000,0.0,0.0,0.0,27.187500000000000,41.718750000000000
2,2023-01-16T09:55:30Z,cpu1,1681269718-129030-Windows_10_22H2,0.0,0.0,33.283582089552240,0.0,4.477611940298507,0.0,0.0,0.0,16.567164179104477,45.671641791044777
3,2023-01-16T09:55:40Z,cpu-total,1681269718-129030-Windows_10_22H2,0.0,0.0,45.631825273010918,0.0,0.000000000000000,0.0,0.0,0.0,35.257410296411855,19.110764430577223
4,2023-01-16T09:55:40Z,cpu0,1681269718-129030-Windows_10_22H2,0.0,0.0,45.638629283489095,0.0,0.155763239875389,0.0,0.0,0.0,38.317757009345797,15.887850467289720


In [12]:
#Tony Sarajavi queries copied from his graana
q = "SELECT mean(total) as total, mean(buffered) as buffered, mean(cached) as cached, mean(total)-mean(free) as non_free, mean(used) as used, mean(total)-mean(available) as non_available  FROM mem WHERE host =~ /1681269718-122700(-|$)/ AND time >= 1673829274000ms and time <= 1673830968000ms GROUP BY time(30s), host ORDER BY asc"
tdf = pd.DataFrame(telegraf_client.query(q).get_points())
tdf.head()

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,time,total,buffered,cached,non_free,used,non_available
0,2023-01-16T00:34:30Z,16105586688.0,0.0,0.0,16105586688.0,1.795305472000000e+09,1.795305472000000e+09
1,2023-01-16T00:35:00Z,16105586688.0,0.0,0.0,16105586688.0,2.040752810666667e+09,2.040752810666666e+09
2,2023-01-16T00:35:30Z,16105586688.0,0.0,0.0,16105586688.0,2.493913770666667e+09,2.493913770666666e+09
3,2023-01-16T00:36:00Z,16105586688.0,0.0,0.0,16105586688.0,2.725780138666667e+09,2.725780138666666e+09
4,2023-01-16T00:36:30Z,16105586688.0,0.0,0.0,16105586688.0,2.993198421333333e+09,2.993198421333334e+09
